In [ ]:
from pybricksdev.connections import BLEPUPConnection
from asyncio import gather, sleep

In [ ]:
train = BLEPUPConnection()
switch = BLEPUPConnection()

In [ ]:
%%file build/train.py

from pybricks import version
from pybricks.hubs import CityHub
from pybricks.pupdevices import ColorDistanceSensor, DCMotor
from pybricks.parameters import Port, Color
from pybricks.tools import wait

wait(2000)
hub = CityHub()
sensor = ColorDistanceSensor(Port.B)
motor = DCMotor(Port.A)
speed = 55
output = "0"
prev_color = "NONE"
if version[1] == "3.0.0b1":
    train_prefix = "1"

# color map
track = Color(h=195, s=43, v=32, name='TRACK')
red = Color(h=358, s=95, v=77, name='RED')
dark_brown = Color(h=352, s=52, v=31, name='DARK_BROWN')
blue = Color(h=220, s=97, v=67, name='BLUE')
light_green = Color(h=50, s=92, v=76, name='LIGHT_GREEN')
light_purple = Color(h=269, s=67, v=70, name='LIGHT_PURPLE')
orange = Color(h=5, s=96, v=93, name='ORANGE')
tan = Color(h=50, s=64, v=79, name='TAN')
light_blue = Color(h=209, s=96, v=83, name='LIGHT_BLUE')

my_colors = (track, Color.GREEN, red, dark_brown, Color.WHITE, blue, light_green, light_purple, orange, tan, light_blue, None)
sensor.detectable_colors(my_colors)

#motor.dc(speed)

while True:
    color = sensor.color()
    if color != prev_color:
        if color == red:
            hub.light.on(red)
            output = train_prefix + "0"
            print(output)
        elif color == blue:
            hub.light.on(blue)
            output = train_prefix + "1"
        elif color == orange:
            hub.light.on(orange)
            output = train_prefix + "2"
            print(output)
            #motor.dc(speed)
        elif color == Color.GREEN:
            motor.stop()
        else:
            output = "0"
        prev_color = color
    wait(1000)

In [ ]:
%%file build/switch.py

from pybricks.hubs import CityHub
from pybricks.pupdevices import DCMotor
from pybricks.parameters import Port, Color
from pybricks.tools import wait
from pybricks.experimental import getchar

hub = CityHub()
motora = DCMotor(Port.A)
#motorb = DCMotor(Port.B)
status = "main"
pulse = 40
power = 100

#wait(2000)

while True:
    #motor.stop()
    try:
        train_input = getchar()
        print(train_input)
        #train_input = train_input[1:]
    except:
        print("switch: didn't get value")
        pass
    else:
        if train_input == 0:
            hub.light.on(Color.MAGENTA)
        elif train_input == 10:
            hub.light.on(Color.RED)
            motora.dc(-power)
            wait(pulse)
            motora.stop()
        elif train_input == 12:
            hub.light.on(Color.WHITE)
            motora.dc(power)
            wait(pulse)
            motora.stop()
    wait(1000)

In [ ]:
async def forwarder(switch, train):
    
    await sleep(5)

    while True: 
        try:
            if len(train.output) > 0:
                line = train.output[-1]
                train.output = []
                try:
                    speed = int(line.decode())
                    print ("pc: ", speed)
                    await switch.write(bytes([speed]))
                except:
                    print("skipped")
                    pass
        except:
            pass
        await sleep(1)

In [ ]:
async def switch_wrapper():
    
    while True:
        try:
            await switch.connect('90:84:2B:01:14:27')
            await switch.run('build/switch.py', print_output=True)
        except:
            print("Switch hub not connected")
            pass
    await sleep(5)

In [ ]:
async def train_wrapper():
    
    while True:
        try:
            await train.connect('90:84:2B:04:C3:81')
            await train.run('build/train.py', print_output=True)
        except:
            print("Train hub not connected")
            pass
    await sleep(5)

In [ ]:
await gather(
    switch_wrapper(),
    train_wrapper(),
    forwarder(switch, train)
)

In [ ]:
# Disconnect (optional)

try:
    await switch.disconnect()
    await train.disconnect()
except:
    pass